# Green's functions

The Green's function of a linear system tells you how it responds to an impulsive forcing ($\delta$). This response depends both on the differential equation but also, importantly, on the geometry and boundary conditions of the problem.

In [ ]:
import numpy as np
from scipy.integrate import solve_bvp, solve_ivp
import scipy as sp
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def deltafunction(t):
    """Approximation to a delta function: a very narrow Gaussian
    normalised such that its integral is 1."""
    delta = .01
    return np.exp(-(t/delta)**2) / (delta * np.sqrt(np.pi))

## Sheet 3, question 2
$$ G'' - \lambda^2 G = \delta(x - \xi) $$

with homogeneous Dirichlet BCs.

In [ ]:
@interact(lamb=widgets.FloatSlider(min=0, max=10, value=1),
          xi=widgets.FloatSlider(min=0, max=1, value=0.5))
def q32demo(lamb, xi):
    def fun(x, y):
        return [y[1], lamb**2 * y[0] + deltafunction(x - xi)]

    def bcs(ya, yb):
        return [ya[0], yb[0]]

    x_guess = np.linspace(0,1)
    y_guess = np.zeros([2, len(x_guess)])
    sol = solve_bvp(fun, bcs, x_guess, y_guess)

    x_plot = np.linspace(0, 1, 1001)
    y_plot = sol.sol(x_plot)
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(x_plot, y_plot[0,:], 'k-',
            [xi], sol.sol([xi])[0,0], 'ko')
    ax.set(xlim=[-0.05, 1.05], ylim=[-0.25, 0.025])
    ax.grid() 
    ax.set(xlabel="x")
    ax.set(ylabel="y")
    plt.show()

## Sheet 3, question 4

$$ y''''  + k^2 y'' = f(t) $$

with homogeneous BCs at $t=0$. WLOG you can take $k=1$ (otherwise rescale time). 

In [ ]:
@interact_manual(tau=widgets.FloatSlider(min=0, max=10, value=0.4))
def q34demo(tau):
    def forcing(t):
        return deltafunction(t - tau)

    def fun(t, y):
        return [y[1], y[2], y[3], -y[2] + forcing(t)]

    # Since the delta function is zero outside of such a narrow
    # region, we need to take a very small step size so that it
    # doesn't get missed.
    sol = solve_ivp(fun, [0, 30], [0, 0, 0, 0], method='BDF', max_step=0.005)
    
    # t_plot = np.linspace(0, 1, 100)
    # y_plot = sol.sol(t_plot)
    # Normalise the guess (just for demonstration)
    
    plt.plot(sol.t, sol.y[0, :], 'k-',
            [tau], [0], 'ko')
    plt.grid()
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()